# Setup

In [1]:
import os
import json
import gc
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from model.setup import setup_model, save_model, load_model, load_tokenizer
from utils.console import isYes, printc, inputc, print_section
from model.training import preprocess_for_training, train_model
from data.gsm8k import prepare_gsm8k_dataset
from data.svamp import prepare_svamp_dataset
from data.arithmetic import prepare_arithmetic_datasets
from evaluation.math_evaluation import evaluate_math_performance
from evaluation.eval_pipeline import eval_model
from constants import MODEL_NAME, INITIAL_SAVE_PATH, TOOL_FINETUNED_SAVE_PATH, DATASET, CHECKPOINTS, TOOL_TRAIN_DATASET_PATH, PURE_TRAIN_DATASET_PATH, EVAL_DATASET_PATH
from data.arithmetic import combine_and_tokenize
import wandb
from datasets import Dataset

c:\Users\user\scoop\apps\anaconda3\current\envs\torch-gpu3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Constants
current_epochs_tool = 1
current_epochs_pure = 1

In [3]:
print_section("Loading Model")
gc.collect()
torch.cuda.empty_cache()
# Try to load from saved path first, if it fails, download from HF
try:
    model, metadata = load_model(os.path.join(CHECKPOINTS, "pretrained", INITIAL_SAVE_PATH))
    tokenizer = load_tokenizer(os.path.join(CHECKPOINTS, "pretrained", INITIAL_SAVE_PATH))
    print("Loaded model from saved path")
except FileNotFoundError:
    print(f"Initial model not found. Setting up from {MODEL_NAME}")
    tokenizer, model, metadata = setup_model(MODEL_NAME)
    save_model(model, tokenizer, os.path.join(CHECKPOINTS, "pretrained", INITIAL_SAVE_PATH))

print_section("Adding Tool Tokens")
tool_tokens = {
    "additional_special_tokens": [
        "<tool:calculator>",
        "</tool>",
    ]
}
num_added = tokenizer.add_special_tokens(tool_tokens)
print(f"Added {num_added} special tokens to the tokenizer")
model.resize_token_embeddings(len(tokenizer))
print(f"Resized model embeddings to {len(tokenizer)} tokens")
print("Special tokens:", tokenizer.all_special_tokens)


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.



Loading Model

Loading model from local path: ./checkpoints\pretrained\qwen-initial
Loaded model from saved path

Adding Tool Tokens

Added 2 special tokens to the tokenizer
Resized model embeddings to 151648 tokens
Special tokens: ['<|endoftext|>', '<tool:calculator>', '</tool>']


In [4]:
# LOAD DATA
print_section("Loading Data")
# Prepare datasets
dataset = prepare_arithmetic_datasets()
train_data = dataset["train_dict"]
test_data = dataset["test_dict"]
train_transformed_data = dataset["train_transformed_dict"]
test_transformed_data = dataset["test_transformed_dict"]
print(train_transformed_data['arithmetic_2da'][3])
print(test_transformed_data['arithmetic_2da'][3])
print(train_transformed_data['arithmetic_2da'][-1])
print(train_transformed_data['arithmetic_2da'][-1])


Loading Data

Processing dataset configuration: arithmetic_1dc
Processing dataset configuration: arithmetic_2da
{'question': 'Question: What is 31 plus 72?\nAnswer:', 'final_answer': '<tool:calculator>31 + 72</tool>'}
{'question': 'Question: What is 12 plus 63?\nAnswer:', 'final_answer': '<tool:calculator>12 + 63</tool>'}
{'question': 'Question: What is 46 plus 53?\nAnswer:', 'final_answer': '<tool:calculator>46 + 53</tool>'}
{'question': 'Question: What is 46 plus 53?\nAnswer:', 'final_answer': '<tool:calculator>46 + 53</tool>'}


# Pretrained

In [6]:
# PRETAINED MODEL EVALUATION
print_section("Pretrained Model Evaluation")
eval_model(MODEL_NAME, DATASET, test_data, model, tokenizer, use_tool=False)

c:\Users\user\scoop\apps\anaconda3\current\envs\torch-gpu3\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\user\scoop\apps\anaconda3\current\envs\torch-gpu3\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



Pretrained Model Evaluation

Evaluating math performance on dataset arithmetic_1dc
Evaluating math performance...
------
Question: What is (4 - 2) + 7?
Answer:
------


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Expected: 9
Model: 9
Correct: 1, Total: 1


------
Question: What is (1 + 8) - 8?
Answer:
------


KeyboardInterrupt: 

# Toolformer

In [ ]:
# TOOLFORMER FINE TUNING TRAINING
print_section("Toolformer Fine Tuning Training")

# Prepare the training data based on dataset type
train_dataset = combine_and_tokenize(train_transformed_data, tokenizer, path=TOOL_TRAIN_DATASET_PATH)

# Create a small evaluation dataset directly instead of using combine_and_tokenize
eval_examples = []
for config_name, config_dataset in test_transformed_data.items():
    # Take at most 5 examples from each configuration
    sample_size = 1
    for i in range(sample_size):
        if isinstance(config_dataset[i], dict):
            eval_examples.append({
                "question": config_dataset[i]["question"],
                "final_answer": config_dataset[i]["final_answer"]
            })

# Create the evaluation dataset directly
eval_dataset = Dataset.from_list(eval_examples)
eval_dataset = eval_dataset.map(
    lambda examples: preprocess_for_training(examples, tokenizer),
    batched=True,
    remove_columns=eval_dataset.column_names
)

print(f"Created evaluation dataset with {len(eval_dataset)} examples for monitoring")

# Load previous model if it exists
try:
    previous_path = os.path.join(CHECKPOINTS, "finetuned", TOOL_FINETUNED_SAVE_PATH)
    tokenizer, model, metadata = load_model(previous_path)
    
    # Get total epochs from metadata
    total_epochs = metadata.get("total_epochs", 0) + current_epochs_tool
    print(f"Continuing training from {metadata.get('total_epochs', 0)} epochs to {total_epochs} epochs")
except FileNotFoundError:
    # Start fresh training
    total_epochs = current_epochs_tool
    print(f"Starting fresh training for {current_epochs_tool} epochs")

# Train the model
model, tokenizer, metadata = train_model(model, tokenizer, train_dataset, num_epochs=current_epochs_tool, eval_dataset=eval_dataset)

# Save with updated epoch count
saved_path = save_model(
    model, 
    tokenizer, 
    os.path.join(CHECKPOINTS, "finetuned", TOOL_FINETUNED_SAVE_PATH),
    epochs=current_epochs_tool,
    total_epochs=total_epochs
)
print(f"Saved fine-tuned model to {saved_path} (Total epochs: {total_epochs})")

In [ ]:
print_section("Latest Checkpoint Evaluation")
model, metadata = load_model(os.path.join(os.curdir, "toolformer_model", "checkpoint-225"))
print_section("Most recent training Model Evaluation")
eval_model(MODEL_NAME, DATASET, test_data, model, tokenizer, use_tool=True)


Latest Checkpoint Evaluation

Loading model from local path: .\toolformer_model\checkpoint-225


c:\Users\user\scoop\apps\anaconda3\current\envs\torch-gpu3\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\user\scoop\apps\anaconda3\current\envs\torch-gpu3\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



Most recent training Model Evaluation

Evaluating math performance on dataset arithmetic_1dc
Evaluating math performance...
------
Question: What is (4 - 2) + 7?
Answer:
------


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


USING TOOLS, PRE-TOOL TEXT: Question: What is (4 - 2) + 7?
Answer:

Answer: 11
--------------------
Expected: 9
Model: 11
Correct: 0, Total: 1


------
Question: What is (1 + 8) - 8?
Answer:
------
USING TOOLS, PRE-TOOL TEXT: Question: What is (1 + 8) - 8?
Answer:

Answer: 1
--------------------
Expected: 1
Model: 1
Correct: 1, Total: 2


------
Question: What is (4 - 3) - 4?
Answer:
------
USING TOOLS, PRE-TOOL TEXT: Question: What is (4 - 3) - 4?
Answer:

Answer: 1
--------------------
Expected: -3
Model: 1
Correct: 1, Total: 3


------
Question: What is (1 + 2) - 7?
Answer:
------


...

USING TOOLS, PRE-TOOL TEXT: Question: What is 7 plus 1?
Answer:

Answer: 8
--------------------
Expected: 8
Model: 8
Correct: 62, Total: 74


------
Question: What is 33 plus 85?
Answer:
------
USING TOOLS, PRE-TOOL TEXT: Question: What is 33 plus 85?
Answer:

Answer: 128
--------------------
Expected: 118
Model: 128
Correct: 62, Total: 75


------
Question: What is 53 plus 48?
Answer:
------


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


USING TOOLS, PRE-TOOL TEXT: Question: What is 53 plus 48?
Answer:

Answer: 101
--------------------
Expected: 101
Model: 101
Correct: 63, Total: 76


------
Question: What is 20 plus 0?
Answer:
------
USING TOOLS, PRE-TOOL TEXT: Question: What is 20 plus 0?
Answer:

Answer: 20
--------------------
Expected: 20
Model: 20
Correct: 64, Total: 77


------
Question: What is 0 plus 6?
Answer:
------


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


USING TOOLS, PRE-TOOL TEXT: Question: What is 0 plus 6?
Answer:

Answer: 6
--------------------
Expected: 6
Model: 6
Correct: 65, Total: 78


------
Question: What is 95 plus 53?
Answer:
------
USING TOOLS, PRE-TOOL TEXT: Question: What is 95 plus 53?
Answer:

Answer: 148
--------------------
Expected: 148
Model: 148
Correct: 66, Total: 79


------
Question: What is 93 plus 27?
Answer:
------


# Pure Fine Tuning

In [ ]:
# PURE FINE TUNING TRAINING
print_section("Pure Fine Tuning Training")

# Prepare the training data based on dataset type
train_dataset = combine_and_tokenize(train_data, tokenizer, path=PURE_TRAIN_DATASET_PATH)

# Load previous model if it exists
try:
    previous_path = os.path.join(CHECKPOINTS, "finetuned", TOOL_FINETUNED_SAVE_PATH)
    tokenizer, model, metadata = load_model(previous_path)
    
    # Get total epochs from metadata
    total_epochs = metadata.get("total_epochs", 0) + current_epochs_pure
    print(f"Continuing training from {metadata.get('total_epochs', 0)} epochs to {total_epochs} epochs")
except FileNotFoundError:
    # Start fresh training
    total_epochs = current_epochs_pure
    print(f"Starting fresh training for {current_epochs_pure} epochs")

# Train the model
model, tokenizer, metadata = train_model(model, tokenizer, train_dataset, num_epochs=current_epochs_pure)

# Save with updated epoch count
saved_path = save_model(
    model, 
    tokenizer, 
    os.path.join(CHECKPOINTS, "finetuned", TOOL_FINETUNED_SAVE_PATH),
    epochs=current_epochs_pure,
    total_epochs=total_epochs
)
print(f"Saved fine-tuned model to {saved_path} (Total epochs: {total_epochs})")